In [0]:
from bs4 import BeautifulSoup
import requests

import numpy as np
import pandas as pd
from pandas.io.json import json_normalize #transform json data into pd df
import json

import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim

In [0]:
 url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [0]:
response = requests.get(url)
data = response.text
soup = BeautifulSoup(data,'html.parser')

In [0]:
S_no = 0
d = dict()

table = soup.find('table')
table_rows = table.find_all('tr')
for tr in table_rows:
  td = tr.find_all('td')
  row = [i.text for i in td]
  #print(row)
  S_no +=1
  d[S_no] = row

In [0]:
d[4]

['M3A\n', 'North York\n', 'Parkwoods\n']

In [0]:
df = pd.DataFrame.from_dict(d,orient='index',columns=['Postal_Code','Borough','Neighborhood'])

In [0]:
df.head()

,Postal_Code,Borough,Neighborhood
1,None,None,None
2,M1A\n,Not assigned\n,\n
3,M2A\n,Not assigned\n,\n
4,M3A\n,North York\n,Parkwoods\n
5,M4A\n,North York\n,Victoria Village\n


In [0]:
df = df.replace('\n','', regex=True)

In [0]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)
df = df.reset_index(drop=True)
df.head()

,Postal_Code,Borough,Neighborhood
0,None,None,None
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,"Regent Park, Harbourfront"
4,M6A,North York,"Lawrence Manor, Lawrence Heights"


In [0]:
df.shape

(104, 3)

In [0]:
df.drop(df.index[0], inplace = True)
df = df.reset_index(drop=True)
df.head()

,Postal_Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [0]:
df.shape

(103, 3)

In [0]:
type(df.iloc[0,1])

str

In [0]:
df['latitude'] = None
df['longitude'] = None
df.head()

,Postal_Code,Borough,Neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,None,None
1,M4A,North York,Victoria Village,None,None
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",None,None
3,M6A,North York,"Lawrence Manor, Lawrence Heights",None,None
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",None,None


In [0]:
for i in range(len(df)):
  address = df.iloc[i,1]+' , Toronto, Ontario'
  geolocator = Nominatim(user_agent="ny_explorer")
  location = geolocator.geocode(address)
  latitude = location.latitude
  longitude = location.longitude
  df.iloc[i,3] = latitude
  df.iloc[i,4] = longitude

In [0]:
df.head()

,Postal_Code,Borough,Neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,43.7543,-79.4491
1,M4A,North York,Victoria Village,43.7543,-79.4491
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6563,-79.3809
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7543,-79.4491
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6563,-79.3809


In [0]:
#Different method of getting coordinates
'''import sys
!{sys.executable} -m pip install geocoder
import geocoder
from geopy.geocoders import arcgis as arcgis'''

'import sys\n!{sys.executable} -m pip install geocoder\nimport geocoder\nfrom geopy.geocoders import arcgis as arcgis'

In [0]:
'''code = 'M5A'
g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
lat_lng_coords = g.latlng
lat_lng_coords'''

"code = 'M5A'\ng = geocoder.arcgis('{}, Toronto, Ontario'.format(code))\nlat_lng_coords = g.latlng\nlat_lng_coords"

In [0]:
df.shape

(103, 5)

In [0]:
print('Dataframe has {} Borough and {} neighborhood.'.format(len(df.Borough.unique()),df.shape[0]))

Dataframe has 10 Borough and 103 neighborhood.


###Using geopy library to visualize the place after getting cordinates

In [0]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="can_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [0]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, borough, neighborhood in zip(df['latitude'], df['longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
#map_toronto

In [0]:
print('=================================================================================================================')

##Define Foursquare Credentials and Version

In [0]:
CLIENT_ID = 'PYANEEI0VEF3DT0UYBXNCSB2JR0U230EJF0NYRKSZMNBSL1H' # your Foursquare ID
CLIENT_SECRET = 'L2B5GO3BHAZPAS21BWWX2K0XMBCMWFW3QS5DDGLVPCJV0VYB' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PYANEEI0VEF3DT0UYBXNCSB2JR0U230EJF0NYRKSZMNBSL1H
CLIENT_SECRET:L2B5GO3BHAZPAS21BWWX2K0XMBCMWFW3QS5DDGLVPCJV0VYB


####Let's explore the first neighborhood in our dataframe.¶

In [0]:
 df.loc[0, 'Neighborhood']

'Parkwoods'

In [0]:
neighborhood_name = df.loc[0, 'Neighborhood'] 
neighborhood_latitude = df.loc[0, 'latitude'] 
neighborhood_longitude = df.loc[0, 'longitude'] 

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7543263, -79.44911696639593.


In [0]:
# Top 100 venues near to the specified neighborhood
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=PYANEEI0VEF3DT0UYBXNCSB2JR0U230EJF0NYRKSZMNBSL1H&client_secret=L2B5GO3BHAZPAS21BWWX2K0XMBCMWFW3QS5DDGLVPCJV0VYB&v=20180604&ll=43.7543263,-79.44911696639593&radius=500&limit=100'

In [0]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eb5120f40a7ea001b03c783'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-5a888f7647f8767d37b92f00-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mediterranean_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1c0941735',
         'name': 'Mediterranean Restaurant',
         'pluralName': 'Mediterranean Restaurants',
         'primary': True,
         'shortName': 'Mediterranean'}],
       'id': '5a888f7647f8767d37b92f00',
       'location': {'address': '832 Sheppard West',
        'cc': 'CA',
        'city': 'North York',
        'country': 'Canada',
        'crossStreet': 'NE corner of Sheppard & Wilmington',
        'distance': 246,
        'formattedAddress': ['832 Sheppard West (NE corner of Sheppard & Wilmin

In [0]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean the json and structure it into a pandas dataframe

In [0]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Grill Gate,Mediterranean Restaurant,43.753123,-79.451690
1,Orly Restaurant & Grill,Middle Eastern Restaurant,43.754493,-79.443507
2,Tim Hortons,Coffee Shop,43.754767,-79.443250
3,Domino's Pizza,Pizza Place,43.753127,-79.450926


In [0]:
print('{} venues were returned by Foursquare near to the specified neighbor'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare near to the specified neighbor


In [0]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [71]:
nearby_venues.head()

,name,categories,lat,lng
0,Grill Gate,Mediterranean Restaurant,43.753123,-79.451690
1,Orly Restaurant & Grill,Middle Eastern Restaurant,43.754493,-79.443507
2,Tim Hortons,Coffee Shop,43.754767,-79.443250
3,Domino's Pizza,Pizza Place,43.753127,-79.450926


In [0]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                 latitudes=df['latitude'],
                                 longitudes=df['longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview
The Danforth West, Ri

In [72]:
toronto_venues.shape

(412, 7)

In [73]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.754326,-79.449117,Grill Gate,43.753123,-79.451690,Mediterranean Restaurant
1,Parkwoods,43.754326,-79.449117,Orly Restaurant & Grill,43.754493,-79.443507,Middle Eastern Restaurant
2,Parkwoods,43.754326,-79.449117,Tim Hortons,43.754767,-79.443250,Coffee Shop
3,Parkwoods,43.754326,-79.449117,Domino's Pizza,43.753127,-79.450926,Pizza Place
4,Victoria Village,43.754326,-79.449117,Grill Gate,43.753123,-79.451690,Mediterranean Restaurant


In [75]:
print('There are {} unique Venue categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 4 unique Venue categories.


In [78]:
toronto_venues.groupby('Neighborhood').count().head()  #Each neighborhood had returned 4 venues

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",4,4,4,4,4,4
"Bathurst Manor, Wilson Heights, Downsview North",4,4,4,4,4,4
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",4,4,4,4,4,4


One-hot encoding

In [82]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.head()

,Coffee Shop,Mediterranean Restaurant,Middle Eastern Restaurant,Pizza Place
0,0,1,0,0
1,0,0,1,0
2,1,0,0,0
3,0,0,0,1
4,0,1,0,0


In [83]:
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
toronto_onehot.head()

,Coffee Shop,Mediterranean Restaurant,Middle Eastern Restaurant,Pizza Place,Neighborhood
0,0,1,0,0,Parkwoods
1,0,0,1,0,Parkwoods
2,1,0,0,0,Parkwoods
3,0,0,0,1,Parkwoods
4,0,1,0,0,Victoria Village


In [88]:
toronto_onehot = toronto_onehot[[toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])]
toronto_onehot.head()

,Neighborhood,Coffee Shop,Mediterranean Restaurant,Middle Eastern Restaurant,Pizza Place
0,Parkwoods,0,1,0,0
1,Parkwoods,0,0,1,0
2,Parkwoods,1,0,0,0
3,Parkwoods,0,0,0,1
4,Victoria Village,0,1,0,0


In [90]:
# group rows by neighborhood taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Coffee Shop,Mediterranean Restaurant,Middle Eastern Restaurant,Pizza Place
0,Agincourt,0.25,0.25,0.25,0.25
1,"Alderwood, Long Branch",0.25,0.25,0.25,0.25
2,"Bathurst Manor, Wilson Heights, Downsview North",0.25,0.25,0.25,0.25
3,Bayview Village,0.25,0.25,0.25,0.25
4,"Bedford Park, Lawrence Manor East",0.25,0.25,0.25,0.25


In [94]:
len(df.Neighborhood.unique())

98

In [91]:
toronto_grouped.shape

(98, 5)

In [96]:
num_top_venues = 5  #top 5 venues of each neighborhood

for hood in toronto_grouped['Neighborhood']:
    print("  ----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

  ----Agincourt----
                       venue  freq
0                Coffee Shop  0.25
1   Mediterranean Restaurant  0.25
2  Middle Eastern Restaurant  0.25
3                Pizza Place  0.25


  ----Alderwood, Long Branch----
                       venue  freq
0                Coffee Shop  0.25
1   Mediterranean Restaurant  0.25
2  Middle Eastern Restaurant  0.25
3                Pizza Place  0.25


  ----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                Coffee Shop  0.25
1   Mediterranean Restaurant  0.25
2  Middle Eastern Restaurant  0.25
3                Pizza Place  0.25


  ----Bayview Village----
                       venue  freq
0                Coffee Shop  0.25
1   Mediterranean Restaurant  0.25
2  Middle Eastern Restaurant  0.25
3                Pizza Place  0.25


  ----Bedford Park, Lawrence Manor East----
                       venue  freq
0                Coffee Shop  0.25
1   Mediterranean Restaurant  0.25
2  Mid

In [0]:
def return_most_common_venues(row, num_top_venues):         # sorting in descending order
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [102]:
# New dataframe to display the top 3 venues for each neighborhood
num_top_venues = 3
indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,Agincourt,Pizza Place,Middle Eastern Restaurant,Mediterranean Restaurant
1,"Alderwood, Long Branch",Pizza Place,Middle Eastern Restaurant,Mediterranean Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Pizza Place,Middle Eastern Restaurant,Mediterranean Restaurant
3,Bayview Village,Pizza Place,Middle Eastern Restaurant,Mediterranean Restaurant
4,"Bedford Park, Lawrence Manor East",Pizza Place,Middle Eastern Restaurant,Mediterranean Restaurant


In [103]:
# clustering the neighborhood into 5 clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  after removing the cwd from sys.path.


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [106]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() 

,Postal_Code,Borough,Neighborhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,M3A,North York,Parkwoods,43.7543,-79.4491,0,Pizza Place,Middle Eastern Restaurant,Mediterranean Restaurant
1,M4A,North York,Victoria Village,43.7543,-79.4491,0,Pizza Place,Middle Eastern Restaurant,Mediterranean Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6563,-79.3809,0,Pizza Place,Middle Eastern Restaurant,Mediterranean Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7543,-79.4491,0,Pizza Place,Middle Eastern Restaurant,Mediterranean Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6563,-79.3809,0,Pizza Place,Middle Eastern Restaurant,Mediterranean Restaurant


In [108]:
map_clusters = folium.Map(location=[43.67635739999999, -79.2930312], zoom_start=11)

# color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['latitude'], toronto_merged['longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [109]:
# CLUSTER 1

toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
0,North York,0,Pizza Place,Middle Eastern Restaurant,Mediterranean Restaurant
1,North York,0,Pizza Place,Middle Eastern Restaurant,Mediterranean Restaurant
2,Downtown Toronto,0,Pizza Place,Middle Eastern Restaurant,Mediterranean Restaurant
3,North York,0,Pizza Place,Middle Eastern Restaurant,Mediterranean Restaurant
4,Downtown Toronto,0,Pizza Place,Middle Eastern Restaurant,Mediterranean Restaurant
...,...,...,...,...,...
98,Etobicoke,0,Pizza Place,Middle Eastern Restaurant,Mediterranean Restaurant
99,Downtown Toronto,0,Pizza Place,Middle Eastern Restaurant,Mediterranean Restaurant
100,East Toronto,0,Pizza Place,Middle Eastern Restaurant,Mediterranean Restaurant
101,Etobicoke,0,Pizza Place,Middle Eastern Restaurant,Mediterranean Restaurant


In [112]:
# CLUSTER 2

toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue


In [113]:
# CLUSTER 3

toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue


In [114]:
# CLUSTER 4

toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue


In [115]:
# CLUSTER 5

toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue
